In [1]:
pip install pandas nLtk

Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from langdetect import detect, DetectorFactory

In [2]:
#download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\paris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
#define stop words
stop_words=set(stopwords.words('english'))

In [4]:
# Load the CSV file of dataset and select only the 'Comments' column
data = pd.read_csv(r'C:\Users\paris\OneDrive\thesis\collect data\TikTok-comments.csv')[['Comments']]

In [5]:
print(data.head())

                                            Comments
0  Jojoba oil instead of olive oil 🙈 jojoba is me...
1        Do you just throw them in the washer after?
2  The magic makeup eraser towels last for years ...
3                      The makeup eraser cloth…10/10
4  esthetician here— try almond oil or jojoba oil...


In [6]:
# Verify the existing columns
print("Columns in DataFrame:", data.columns)

Columns in DataFrame: Index(['Comments'], dtype='object')


In [7]:
#Lemmatization, reduce words to their base or root form
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\paris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\paris\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
# Define custom stop words excluding sentiment-critical words like "not"

custom_stop_words = stop_words.union({"hii", "day", "bar", "ill", "weather", "new", "sound", "see", "well", "im", "ive", 
   "also", "youre", "win", "thats", "send", "dm", "mascara", "laurence", "nars", "pp", "ao", "de", "e", "xx", "xxx", "get",
   "quel", "savadogo", "banilla", "w", "hiii", "hiiiii"
 
}).difference({"able", "affected", "affecting", "affects", "after", "again", "against", "ain't", "all", "always",
    "anyone", "anybody", "aren't", "awfully", "best", "better", "definitely", "end", "ending", "fairly", "will", "have this",
    "hopefully", "however", "but", "importance", "important", "information", "instead", "interest", "isn't", "have product",
    "less", "like", "liked", "mustn't", "necessary", "needn't", "ok", "okay", "shouldn't", "suggest", 
    "unfortunately", "use", "used", "useful", "using", "want", "wants", "wasn't", "won't", "wouldn't", "not", "no", "never"})
custom_stop_words = {word.lower() for word in custom_stop_words}




lemmatizer = WordNetLemmatizer()

# Function to map Penn Treebank POS tags to WordNet POS tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun



def clean_text(text):
#1. remove URLs, Mentions-Tags-
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\s+|www\s+', '', text)
#2. remove special charachters and numbers,but keep question marks-sentiment-critical
    text = re.sub(r'[^a-zA-Z\s?]','',text)
#3. convert to lowercase- Text normalization
    text = text.lower()
#4. Tokenizw and remove stop words
    tokens = word_tokenize(text)
    filtered_words = [word for word in tokens if word not in custom_stop_words]
    # Join words back into one string
    return ' '.join(filtered_words)


#Apply function to each text in the DataFrame
data['Cleaned_Comments'] = data['Comments'].apply(clean_text)
print(data[['Comments', 'Cleaned_Comments']])



                                               Comments  \
0     Jojoba oil instead of olive oil 🙈 jojoba is me...   
1           Do you just throw them in the washer after?   
2     The magic makeup eraser towels last for years ...   
3                         The makeup eraser cloth…10/10   
4     esthetician here— try almond oil or jojoba oil...   
...                                                 ...   
2358                                 Thank you yes!!!!!   
2359                          We have new mascara too!!   
2360                                  Pacifica is IT✨✨✨   
2361  Elf concealer has forever chemicals in it if s...   
2362            could you please share alternatives? <3   

                                       Cleaned_Comments  
0     jojoba oil instead olive oil jojoba meant clea...  
1                                  throw washer after ?  
2      magic makeup eraser towels last years need water  
3                                   makeup eraser cloth  
4

In [9]:
# Print the first few stop words to confirm
print(list(custom_stop_words)[:30])  # Debugging: Check stop words

['which', 'w', 've', 'from', 'as', 'shouldn', 'their', 'nor', 'more', 'for', 'where', 'shan', 'his', 'because', 'mustn', 'only', 'haven', "shan't", 'get', 'hiii', 'how', 'any', 'some', 'weren', 'who', 'o', "you'd", 'see', 'couldn', 'had']


In [10]:
from nltk.corpus import wordnet 
# Lemmatize text
def lemmatize_text(text):
    tokens = word_tokenize(text)  # Tokenize cleaned text
    pos_tags = pos_tag(tokens)  # Get POS tags
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags
    ]
    return ' '.join(lemmatized_words)  # Return lemmatized text



In [11]:
data['Cleaned_Comments'] = data['Comments'].apply(clean_text)
data['Lemmatized_Comments'] = data['Cleaned_Comments'].apply(lemmatize_text)

print(data[['Comments', 'Cleaned_Comments', 'Lemmatized_Comments']])


                                               Comments  \
0     Jojoba oil instead of olive oil 🙈 jojoba is me...   
1           Do you just throw them in the washer after?   
2     The magic makeup eraser towels last for years ...   
3                         The makeup eraser cloth…10/10   
4     esthetician here— try almond oil or jojoba oil...   
...                                                 ...   
2358                                 Thank you yes!!!!!   
2359                          We have new mascara too!!   
2360                                  Pacifica is IT✨✨✨   
2361  Elf concealer has forever chemicals in it if s...   
2362            could you please share alternatives? <3   

                                       Cleaned_Comments  \
0     jojoba oil instead olive oil jojoba meant clea...   
1                                  throw washer after ?   
2      magic makeup eraser towels last years need water   
3                                   makeup eraser cloth

In [12]:
# Function to detect and keep only English comments
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False  # Default to non-English if detection fails

In [13]:
# Apply cleaning and lemmatization to the DataFrame
data['Cleaned_Comments'] = data['Comments'].apply(clean_text)

#  Remove rows containing only question marks or empty strings
data = data[~data['Cleaned_Comments'].str.fullmatch(r'[\s?]*')]  # Remove rows with only ?, whitespace, or empty

data['Lemmatized_Comments'] = data['Cleaned_Comments'].apply(lemmatize_text)

print(data[['Comments', 'Cleaned_Comments', 'Lemmatized_Comments']])

                                               Comments  \
0     Jojoba oil instead of olive oil 🙈 jojoba is me...   
1           Do you just throw them in the washer after?   
2     The magic makeup eraser towels last for years ...   
3                         The makeup eraser cloth…10/10   
4     esthetician here— try almond oil or jojoba oil...   
...                                                 ...   
2357                                        I love elf.   
2358                                 Thank you yes!!!!!   
2360                                  Pacifica is IT✨✨✨   
2361  Elf concealer has forever chemicals in it if s...   
2362            could you please share alternatives? <3   

                                       Cleaned_Comments  \
0     jojoba oil instead olive oil jojoba meant clea...   
1                                  throw washer after ?   
2      magic makeup eraser towels last years need water   
3                                   makeup eraser cloth

In [16]:
#  Clean comments
#data['Cleaned_Comments'] = data['Comments'].apply(clean_text)

# Remove rows containing only question marks or empty strings
#data = data[~data['Cleaned_Comments'].str.fullmatch(r'[\s?]*')]  # Remove rows with only ?, whitespace, or empty

# Remove non-English comments
#data = data[data['Cleaned_Comments'].apply(is_english)]

#  Lemmatize comments
#data['Lemmatized_Comments'] = data['Cleaned_Comments'].apply(lemmatize_text)

# Display results
#print(data[['Comments', 'Cleaned_Comments', 'Lemmatized_Comments']])

In [14]:
# Identify duplicate rows
duplicates = data[data.duplicated()]

print(f"Number of duplicate rows based on lemmatized comments: {duplicates.shape[0]}")
print(duplicates[['Comments', 'Cleaned_Comments', 'Lemmatized_Comments']])

Number of duplicate rows based on lemmatized comments: 8
                           Comments         Cleaned_Comments  \
210         Good to know thank you!          good know thank   
1140              Thanks for this!!                   thanks   
1372      Is Clinique cruelty free?  clinique cruelty free ?   
1469  Is the ordinary cruelty free?  ordinary cruelty free ?   
1635                            Yep                      yep   
1860                       Amazing!                  amazing   
2111        So glad to hear that! 💚                glad hear   
2212                      thank you                    thank   

          Lemmatized_Comments  
210           good know thank  
1140                   thanks  
1372  clinique cruelty free ?  
1469  ordinary cruelty free ?  
1635                      yep  
1860                    amaze  
2111                glad hear  
2212                    thank  


In [15]:
# Remove duplicates but keep single-word comments
data_cleaned = data[~(data.duplicated(subset=['Lemmatized_Comments']) & 
                      (data['Lemmatized_Comments'].str.split().str.len() > 1))]

# Check the shape of the cleaned DataFrame
print(f"Shape of the cleaned DataFrame (after removing duplicates but keeping single-word comments): {data_cleaned.shape}")

Shape of the cleaned DataFrame (after removing duplicates but keeping single-word comments): (2237, 3)


In [16]:
# Remove rows where 'Lemmatized_Comments' is empty or only contains whitespace, but avoid reformatting issues
data = data[data['Lemmatized_Comments'].astype(bool)]  # Keep only rows with non-empty strings

# Check the resulting DataFrame to ensure formatting is preserved
print(data[['Comments', 'Cleaned_Comments', 'Lemmatized_Comments']])

                                               Comments  \
0     Jojoba oil instead of olive oil 🙈 jojoba is me...   
1           Do you just throw them in the washer after?   
2     The magic makeup eraser towels last for years ...   
3                         The makeup eraser cloth…10/10   
4     esthetician here— try almond oil or jojoba oil...   
...                                                 ...   
2357                                        I love elf.   
2358                                 Thank you yes!!!!!   
2360                                  Pacifica is IT✨✨✨   
2361  Elf concealer has forever chemicals in it if s...   
2362            could you please share alternatives? <3   

                                       Cleaned_Comments  \
0     jojoba oil instead olive oil jojoba meant clea...   
1                                  throw washer after ?   
2      magic makeup eraser towels last years need water   
3                                   makeup eraser cloth

In [18]:
# Detect and store non-English rows
#non_english_rows = data[~data['Cleaned_Comments'].apply(is_english)]

# Display non-English rows
#print(non_english_rows[['Comments', 'Cleaned_Comments']])

In [17]:
data.to_csv(r'C:\Users\paris\OneDrive\thesis\collect data\Cleaned_TikTok_comments3.csv', index=False)


In [19]:
# Function to extract words by part of speech (POS)
def extract_pos(text, pos_list):
    tokens = word_tokenize(text)  # Tokenize the text
    tagged_tokens = pos_tag(tokens)  # Part-of-speech tagging
    return [word for word, tag in tagged_tokens if tag in pos_list]

# Define POS tags for adjectives, nouns, and verbs
adjective_tags = ['JJ', 'JJR', 'JJS']
noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']
verb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# Extract adjectives, nouns, and verbs from the dataset
data['adjectives'] = data['Lemmatized_Comments'].apply(lambda x: extract_pos(x, adjective_tags))
data['nouns'] = data['Lemmatized_Comments'].apply(lambda x: extract_pos(x, noun_tags))
data['verbs'] = data['Lemmatized_Comments'].apply(lambda x: extract_pos(x, verb_tags))

# Display the DataFrame with extracted parts of speech
print(data)

                                               Comments  \
0     Jojoba oil instead of olive oil 🙈 jojoba is me...   
1           Do you just throw them in the washer after?   
2     The magic makeup eraser towels last for years ...   
3                         The makeup eraser cloth…10/10   
4     esthetician here— try almond oil or jojoba oil...   
...                                                 ...   
2357                                        I love elf.   
2358                                 Thank you yes!!!!!   
2360                                  Pacifica is IT✨✨✨   
2361  Elf concealer has forever chemicals in it if s...   
2362            could you please share alternatives? <3   

                                       Cleaned_Comments  \
0     jojoba oil instead olive oil jojoba meant clea...   
1                                  throw washer after ?   
2      magic makeup eraser towels last years need water   
3                                   makeup eraser cloth

In [21]:
data.to_csv(r'C:\Users\paris\OneDrive\thesis\collect data\extracted_pos_results.csv', index=False)